In [9]:
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# For Colab
code_dir = "/content/drive/MyDrive/ClimateLens"
data_dir = "/content/drive/MyDrive/ClimateLens/data"

print('Paths set:', bool(code_dir), bool(data_dir))

directories = {
    "emotions": Path(code_dir) / "visualizations" / "emotions",
    "sentiments": Path(code_dir) / "visualizations" / "sentiments",
}

for path in directories.values():
    os.makedirs(path, exist_ok=True)

emotion_dir = directories.get("emotions")
sentiment_dir = directories.get("sentiments")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Paths set: True True


Emotion
*   Word clouds to represent overall emotional distribution

+ Time-series visualization to show how emotions change over time


In [10]:
# Upload files directly to Colab session
from google.colab import files
import io

print("Please upload your CSV files:")
uploaded = files.upload()

# Get the actual uploaded filenames (handles duplicates like "(2)")
uploaded_files = list(uploaded.keys())
print(f"\nUploaded files: {uploaded_files}")

# Find Twitter and Reddit files
twitter_file = [f for f in uploaded_files if 'twitter' in f.lower()][0]
reddit_file = [f for f in uploaded_files if 'reddit' in f.lower() or 'anticonsumption' in f.lower()][0]

# Load the files
twitter_df = pd.read_csv(io.BytesIO(uploaded[twitter_file]))
reddit_df = pd.read_csv(io.BytesIO(uploaded[reddit_file]))

# Create dictionary for easy iteration
dfs = {
    'Twitter': twitter_df,
    'Reddit': reddit_df
}

print(f"Twitter data loaded: {len(twitter_df)} rows")
print(f"Reddit data loaded: {len(reddit_df)} rows")

Please upload your CSV files:


Saving filtered_anticonsumption_comments.csv to filtered_anticonsumption_comments (5).csv
Saving climate_twitter_sample.csv to climate_twitter_sample (5).csv

Uploaded files: ['filtered_anticonsumption_comments (5).csv', 'climate_twitter_sample (5).csv']
Twitter data loaded: 2736 rows
Reddit data loaded: 2736 rows


Sentiment Visualizations
1. Distribution as Pie Charts
2. Sentiment Probability Histograms
3. Sentiment Probability Violin Plots

In [14]:
sent_dist = Path(sentiment_dir) / "pie charts"
sent_vio = Path(sentiment_dir) / "violin distributions"
sent_his = Path(sentiment_dir) / "probability histograms"

os.makedirs(sent_dist, exist_ok=True)
os.makedirs(sent_vio, exist_ok=True)
os.makedirs(sent_his, exist_ok=True)

In [15]:
save_dir = sent_dist

def create_pie_plot(df,title):
    sentiment_counts = df['sentiment_label'].value_counts()
    sentiment_counts.plot(
        kind='pie',
        autopct='%1.1f%%',
        colors=['skyblue', 'lightcoral', 'lightgreen'],
        labels=sentiment_counts.index
    )
    plt.title(title)

for name, df in dfs.items(): # saving each seperately
    plt.figure(figsize=(6, 6))
    create_pie_plot(df, name)
    file_path = os.path.join(save_dir, f"{name}_sentiment_pie.png")
    plt.savefig(file_path, dpi=300)
    plt.close()
    print(f"Saved: {file_path}")

# one big combined visual
n_datasets = len(dfs)
plt.figure(figsize=(6, 4 * n_datasets))

for idx, (name, df) in enumerate(dfs.items(), start=1):
    plt.subplot(n_datasets, 1, idx)
    create_pie_plot(df, name)

plt.tight_layout()
combined_path = os.path.join(save_dir, "all_sentiment_pies.png")
plt.savefig(combined_path, dpi=300)
plt.close()

print(f"Combined visualization saved to: {combined_path}")

KeyError: 'sentiment_label'

<Figure size 600x600 with 0 Axes>

## **Sentiment Probability Distribution by Sentiment Label**

In [16]:
save_dir=sent_vio

def create_violin_plot(df, title):
    sns.violinplot(
        data=df,
        x='sentiment_proba',
        y='sentiment_label',
        inner='box',
        palette='husl',
        hue='sentiment_label'
    )
    sns.despine(top=True, right=True, bottom=True, left=True)
    plt.title(title)

for name, df in dfs.items():
    plt.figure(figsize=(8, 6))
    create_violin_plot(df, name)
    file_path = os.path.join(save_dir, f"{name}_sentiment_violin.png")
    plt.savefig(file_path, dpi=300)
    plt.close()
    print(f"Saved: {file_path}")

n_datasets = len(dfs)
plt.figure(figsize=(8, 6 * n_datasets))

for idx, (name, df) in enumerate(dfs.items(), start=1):
    plt.subplot(n_datasets, 1, idx)
    create_violin_plot(df, name)

plt.tight_layout()
combined_path = os.path.join(save_dir, "all_sentiment_violins.png")
plt.savefig(combined_path, dpi=300)
plt.close()

print(f"Combined visualization saved to: {combined_path}")

ValueError: Could not interpret value `sentiment_proba` for `x`. An entry with this name does not appear in `data`.

<Figure size 800x600 with 0 Axes>

In [17]:
save_dir=sent_his

def create_histplot(df, title):
    sns.histplot(
        x='sentiment_proba',
        hue='sentiment_label',
        data=df,
        element='step'
    )
    plt.title(title)
    plt.xlabel('Sentiment Probability')
    plt.ylabel('Frequency')

for name, df in dfs.items():
    plt.figure(figsize=(12, 5))
    create_histplot(df, name)
    file_path = os.path.join(save_dir, f"{name}_sentiment_hist.png")
    plt.savefig(file_path, dpi=300)
    plt.close()
    print(f"Saved: {file_path}")

n_datasets = len(dfs)
plt.figure(figsize=(12, 4 * n_datasets))

for idx, (name, df) in enumerate(dfs.items(), start=1):
    plt.subplot(n_datasets, 1, idx)
    create_histplot(df, name)

plt.tight_layout()
combined_path = os.path.join(save_dir, "all_sentiment_hists.png")
plt.savefig(combined_path, dpi=300)
plt.close()

print(f"Combined visualization saved to: {combined_path}")

ValueError: Could not interpret value `sentiment_proba` for `x`. An entry with this name does not appear in `data`.

<Figure size 1200x500 with 0 Axes>

In [1]:
# Install required packages
!pip install -q wordcloud plotly transformers torch kaleido
print(" Packages installed")

 Packages installed


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Visualization
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# NLP & Transformers
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch

print(" Libraries imported")

 Libraries imported


In [3]:
# Load emotion detection model
print("Loading emotion detection model...")
model_name = "j-hartmann/emotion-english-distilroberta-base"

# Check for GPU
device = 0 if torch.cuda.is_available() else -1
print(f"Using device: {'GPU' if device == 0 else 'CPU'}")

# Initialize pipeline
emotion_classifier = pipeline(
    "text-classification",
    model=model_name,
    tokenizer=model_name,
    device=device,
    top_k=None  # Return all emotion scores
)

print("Model loaded successfully")
print(f"Emotions detected: anger, disgust, fear, joy, neutral, sadness, surprise")

Loading emotion detection model...
Using device: CPU


Device set to use cpu


Model loaded successfully
Emotions detected: anger, disgust, fear, joy, neutral, sadness, surprise


In [7]:
def detect_emotions_batch(texts, batch_size=32):
    """
    Detect emotions for a list of texts with batch processing.

    Args:
        texts: List of text strings
        batch_size: Number of texts to process at once

    Returns:
        DataFrame with emotion labels and scores
    """
    results = []

    # Process in batches
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]

        # Truncate long texts (model limit: 512 tokens)
        batch = [text[:512] if isinstance(text, str) else "" for text in batch]

        # Get predictions
        predictions = emotion_classifier(batch)

        # Extract top emotion for each text
        for pred in predictions:
            # pred is a list of dicts with 'label' and 'score'
            top_emotion = max(pred, key=lambda x: x['score'])

            # Create dict with all emotion scores
            emotion_scores = {item['label']: item['score'] for item in pred}

            results.append({
                'emotion_label': top_emotion['label'],
                'emotion_confidence': top_emotion['score'],
                **emotion_scores  # Add individual emotion scores
            })

        # Progress indicator
        if (i // batch_size) % 10 == 0:
            print(f"Processed {min(i+batch_size, len(texts))}/{len(texts)} texts...")

    return pd.DataFrame(results)

print(" Emotion detection function ready")

 Emotion detection function ready


In [11]:
print("="*60)
print("DETECTING EMOTIONS - This may take a few minutes...")
print("="*60)

# Twitter emotions
print("\nProcessing Twitter data...")
twitter_emotions = detect_emotions_batch(twitter_df['cleaned_text'].fillna('').tolist())
twitter_df = pd.concat([twitter_df, twitter_emotions], axis=1)
print(f" Twitter: {len(twitter_df)} rows processed")
print(f" Emotion distribution:\n{twitter_df['emotion_label'].value_counts()}\n")

# Reddit emotions
print("\n Processing Reddit data...")
reddit_emotions = detect_emotions_batch(reddit_df['cleaned_text'].fillna('').tolist())
reddit_df = pd.concat([reddit_df, reddit_emotions], axis=1)
print(f" Reddit: {len(reddit_df)} rows processed")
print(f" Emotion distribution:\n{reddit_df['emotion_label'].value_counts()}\n")

# Update dfs dictionary
dfs = {
    'Twitter': twitter_df,
    'Reddit': reddit_df
}

print("="*60)
print(" EMOTION DETECTION COMPLETE")
print("="*60)

DETECTING EMOTIONS - This may take a few minutes...

Processing Twitter data...
Processed 32/2736 texts...
Processed 352/2736 texts...
Processed 672/2736 texts...
Processed 992/2736 texts...
Processed 1312/2736 texts...
Processed 1632/2736 texts...
Processed 1952/2736 texts...
Processed 2272/2736 texts...
Processed 2592/2736 texts...
 Twitter: 2736 rows processed
 Emotion distribution:
emotion_label
neutral     1431
fear         309
joy          301
sadness      265
anger        216
disgust      109
surprise     105
Name: count, dtype: int64


 Processing Reddit data...
Processed 32/2736 texts...
Processed 352/2736 texts...
Processed 672/2736 texts...
Processed 992/2736 texts...
Processed 1312/2736 texts...
Processed 1632/2736 texts...
Processed 1952/2736 texts...
Processed 2272/2736 texts...
Processed 2592/2736 texts...
 Reddit: 2736 rows processed
 Emotion distribution:
emotion_label
anger       822
neutral     631
sadness     414
fear        323
joy         310
surprise    168
disgu

In [12]:
def create_emotion_wordclouds(df, dataset_name, save_dir):
    """
    Create word clouds for each emotion and save as HTML.

    Args:
        df: DataFrame with 'cleaned_text' and 'emotion_label' columns
        dataset_name: Name for the dataset (e.g., 'Twitter', 'Reddit')
        save_dir: Directory to save HTML files
    """
    emotions = df['emotion_label'].unique()

    # Color schemes for emotions
    emotion_colors = {
        'anger': 'Reds',
        'disgust': 'Greens',
        'fear': 'Purples',
        'joy': 'YlOrRd',
        'neutral': 'Greys',
        'sadness': 'Blues',
        'surprise': 'Oranges'
    }

    print(f"\n Creating word clouds for {dataset_name}...")

    # Create word cloud for each emotion
    for emotion in sorted(emotions):
        emotion_texts = df[df['emotion_label'] == emotion]['cleaned_text'].fillna('')
        combined_text = ' '.join(emotion_texts.tolist())

        if len(combined_text.strip()) == 0:
            print(f"  ⚠️  Skipping {emotion} - no text data")
            continue

        # Generate word cloud
        wordcloud = WordCloud(
            width=1200,
            height=600,
            background_color='white',
            colormap=emotion_colors.get(emotion, 'viridis'),
            max_words=100,
            relative_scaling=0.5,
            min_font_size=10
        ).generate(combined_text)

        # Create matplotlib figure
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.imshow(wordcloud, interpolation='bilinear')
        ax.axis('off')
        ax.set_title(f'{dataset_name} - {emotion.capitalize()} Emotion Word Cloud',
                     fontsize=16, fontweight='bold', pad=20)

        # Save as PNG
        filename = f"{dataset_name.lower()}_{emotion}_wordcloud.png"
        filepath = os.path.join(save_dir, filename)
        plt.tight_layout()
        plt.savefig(filepath, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close()

        print(f"   Saved: {filename}")

    # Create combined word cloud (all emotions)
    all_text = ' '.join(df['cleaned_text'].fillna('').tolist())
    wordcloud_all = WordCloud(
        width=1200,
        height=600,
        background_color='white',
        colormap='viridis',
        max_words=150,
        relative_scaling=0.5,
        min_font_size=10
    ).generate(all_text)

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.imshow(wordcloud_all, interpolation='bilinear')
    ax.axis('off')
    ax.set_title(f'{dataset_name} - All Emotions Combined',
                 fontsize=16, fontweight='bold', pad=20)

    filename = f"{dataset_name.lower()}_all_emotions_wordcloud.png"
    filepath = os.path.join(save_dir, filename)
    plt.tight_layout()
    plt.savefig(filepath, dpi=150, bbox_inches='tight', facecolor='white')
    plt.close()

    print(f"   Saved: {filename}")
    print(f" {dataset_name} word clouds complete\n")

print(" Word cloud function ready")

 Word cloud function ready


In [13]:
def create_emotion_timeseries(df, dataset_name, time_column, save_dir):
    """
    Create interactive time-series visualization of emotion trends.

    Args:
        df: DataFrame with emotion data
        dataset_name: Name for the dataset
        time_column: Name of the time column
        save_dir: Directory to save HTML files
    """
    print(f"\n Creating time-series for {dataset_name}...")

    # Prepare time column
    df_plot = df.copy()

    if dataset_name == 'Twitter':
        # Parse Twitter datetime string
        df_plot['datetime'] = pd.to_datetime(df_plot[time_column])
        # Use hourly aggregation for short time span
        df_plot['time_period'] = df_plot['datetime'].dt.floor('H')
        time_format = '%Y-%m-%d %H:%M'
    else:  # Reddit
        # Convert unix timestamp to datetime
        df_plot['datetime'] = pd.to_datetime(df_plot[time_column], unit='s')
        # Use monthly aggregation for long time span
        df_plot['time_period'] = df_plot['datetime'].dt.to_period('M').dt.to_timestamp()
        time_format = '%Y-%m'

    # Count emotions by time period
    emotion_counts = df_plot.groupby(['time_period', 'emotion_label']).size().reset_index(name='count')

    # Calculate percentages
    total_by_period = emotion_counts.groupby('time_period')['count'].transform('sum')
    emotion_counts['percentage'] = (emotion_counts['count'] / total_by_period * 100).round(2)

    # Create interactive plot with plotly
    fig = go.Figure()

    emotions = sorted(emotion_counts['emotion_label'].unique())

    # Color mapping for emotions
    emotion_colors_map = {
        'anger': '#d62728',      # red
        'disgust': '#2ca02c',    # green
        'fear': '#9467bd',       # purple
        'joy': '#ff7f0e',        # orange
        'neutral': '#7f7f7f',    # gray
        'sadness': '#1f77b4',    # blue
        'surprise': '#e377c2'    # pink
    }

    for emotion in emotions:
        emotion_data = emotion_counts[emotion_counts['emotion_label'] == emotion]

        fig.add_trace(go.Scatter(
            x=emotion_data['time_period'],
            y=emotion_data['percentage'],
            name=emotion.capitalize(),
            mode='lines+markers',
            line=dict(color=emotion_colors_map.get(emotion, '#000000'), width=2),
            marker=dict(size=6),
            hovertemplate=f'<b>{emotion.capitalize()}</b><br>' +
                         'Date: %{x}<br>' +
                         'Percentage: %{y:.1f}%<br>' +
                         '<extra></extra>'
        ))

    # Update layout
    fig.update_layout(
        title=dict(
            text=f'{dataset_name} - Emotion Trends Over Time',
            font=dict(size=20, family='Arial Black')
        ),
        xaxis_title='Time Period',
        yaxis_title='Percentage of Posts (%)',
        hovermode='x unified',
        height=600,
        width=1200,
        template='plotly_white',
        legend=dict(
            title='Emotions',
            orientation='v',
            yanchor='top',
            y=1,
            xanchor='left',
            x=1.02
        ),
        margin=dict(l=60, r=150, t=80, b=60)
    )

    # Save as HTML
    filename = f"{dataset_name.lower()}_emotion_timeseries.html"
    filepath = os.path.join(save_dir, filename)
    fig.write_html(filepath)

    print(f"  Saved: {filename}")

    # Also create absolute count version
    fig_count = go.Figure()

    for emotion in emotions:
        emotion_data = emotion_counts[emotion_counts['emotion_label'] == emotion]

        fig_count.add_trace(go.Scatter(
            x=emotion_data['time_period'],
            y=emotion_data['count'],
            name=emotion.capitalize(),
            mode='lines+markers',
            line=dict(color=emotion_colors_map.get(emotion, '#000000'), width=2),
            marker=dict(size=6),
            stackgroup='one',  # Stack the areas
            hovertemplate=f'<b>{emotion.capitalize()}</b><br>' +
                         'Date: %{x}<br>' +
                         'Count: %{y}<br>' +
                         '<extra></extra>'
        ))

    fig_count.update_layout(
        title=dict(
            text=f'{dataset_name} - Emotion Counts Over Time (Stacked)',
            font=dict(size=20, family='Arial Black')
        ),
        xaxis_title='Time Period',
        yaxis_title='Number of Posts',
        hovermode='x unified',
        height=600,
        width=1200,
        template='plotly_white',
        legend=dict(
            title='Emotions',
            orientation='v',
            yanchor='top',
            y=1,
            xanchor='left',
            x=1.02
        ),
        margin=dict(l=60, r=150, t=80, b=60)
    )

    filename_count = f"{dataset_name.lower()}_emotion_timeseries_stacked.html"
    filepath_count = os.path.join(save_dir, filename_count)
    fig_count.write_html(filepath_count)

    print(f"  Saved: {filename_count}")
    print(f" {dataset_name} time-series complete\n")

print(" Time-series function ready")

 Time-series function ready


In [14]:
print("="*60)
print(" GENERATING EMOTION VISUALIZATIONS")
print("="*60)

# Generate word clouds for both datasets
for name, df in dfs.items():
    create_emotion_wordclouds(df, name, emotion_dir)

# Generate time-series for both datasets
create_emotion_timeseries(twitter_df, 'Twitter', 'created_at', emotion_dir)
create_emotion_timeseries(reddit_df, 'Reddit', 'created_utc', emotion_dir)

print("="*60)
print(" ALL VISUALIZATIONS GENERATED")
print("="*60)
print(f"\n Files saved to: {emotion_dir}")
print("\nGenerated files:")
print("  Word Clouds (PNG):")
print("    - twitter_[emotion]_wordcloud.png (7 files)")
print("    - reddit_[emotion]_wordcloud.png (7 files)")
print("  Time-Series (HTML):")
print("    - twitter_emotion_timeseries.html")
print("    - twitter_emotion_timeseries_stacked.html")
print("    - reddit_emotion_timeseries.html")
print("    - reddit_emotion_timeseries_stacked.html")

 GENERATING EMOTION VISUALIZATIONS

 Creating word clouds for Twitter...
   Saved: twitter_anger_wordcloud.png
   Saved: twitter_disgust_wordcloud.png
   Saved: twitter_fear_wordcloud.png
   Saved: twitter_joy_wordcloud.png
   Saved: twitter_neutral_wordcloud.png
   Saved: twitter_sadness_wordcloud.png
   Saved: twitter_surprise_wordcloud.png
   Saved: twitter_all_emotions_wordcloud.png
 Twitter word clouds complete


 Creating word clouds for Reddit...
   Saved: reddit_anger_wordcloud.png
   Saved: reddit_disgust_wordcloud.png
   Saved: reddit_fear_wordcloud.png
   Saved: reddit_joy_wordcloud.png
   Saved: reddit_neutral_wordcloud.png
   Saved: reddit_sadness_wordcloud.png
   Saved: reddit_surprise_wordcloud.png
   Saved: reddit_all_emotions_wordcloud.png
 Reddit word clouds complete


 Creating time-series for Twitter...
  Saved: twitter_emotion_timeseries.html
  Saved: twitter_emotion_timeseries_stacked.html
 Twitter time-series complete


 Creating time-series for Reddit...
  Saved

In [17]:
def create_summary_report(dfs, save_dir):
    """Create an HTML summary report of emotion analysis."""

    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Emotion Analysis Summary</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 40px; background-color: #f5f5f5; }}
            h1 {{ color: #2c3e50; border-bottom: 3px solid #3498db; padding-bottom: 10px; }}
            h2 {{ color: #34495e; margin-top: 30px; }}
            .dataset {{ background-color: white; padding: 20px; margin: 20px 0; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }}
            table {{ border-collapse: collapse; width: 100%; margin: 20px 0; }}
            th, td {{ border: 1px solid #ddd; padding: 12px; text-align: left; }}
            th {{ background-color: #3498db; color: white; }}
            tr:nth-child(even) {{ background-color: #f2f2f2; }}
            .metric {{ display: inline-block; margin: 10px 20px 10px 0; }}
            .metric-value {{ font-size: 24px; font-weight: bold; color: #3498db; }}
            .metric-label {{ color: #7f8c8d; font-size: 14px; }}
        </style>
    </head>
    <body>
        <h1> Emotion Analysis Summary Report</h1>
        <p><strong>Generated:</strong> {timestamp}</p>
    """.format(timestamp=datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

    for name, df in dfs.items():
        emotion_dist = df['emotion_label'].value_counts()
        total = len(df)

        html_content += f"""
        <div class="dataset">
            <h2> {name} Dataset</h2>
            <div class="metric">
                <div class="metric-value">{total:,}</div>
                <div class="metric-label">Total Posts</div>
            </div>
            <div class="metric">
                <div class="metric-value">{len(emotion_dist)}</div>
                <div class="metric-label">Unique Emotions</div>
            </div>
            <div class="metric">
                <div class="metric-value">{emotion_dist.iloc[0]:,}</div>
                <div class="metric-label">Most Common: {emotion_dist.index[0].capitalize()}</div>
            </div>

            <h3>Emotion Distribution</h3>
            <table>
                <tr>
                    <th>Emotion</th>
                    <th>Count</th>
                    <th>Percentage</th>
                </tr>
        """

        for emotion, count in emotion_dist.items():
            percentage = (count / total * 100)
            html_content += f"""
                <tr>
                    <td><strong>{emotion.capitalize()}</strong></td>
                    <td>{count:,}</td>
                    <td>{percentage:.2f}%</td>
                </tr>
            """

        html_content += """
            </table>
        </div>
        """

    html_content += """
        <div class="dataset">
            <h2> Generated Files</h2>
            <h3>Word Clouds (PNG)</h3>
            <ul>
                <li>Individual emotion word clouds for Twitter (7 files)</li>
                <li>Individual emotion word clouds for Reddit (7 files)</li>
            </ul>
            <h3>Time-Series (HTML - Interactive)</h3>
            <ul>
                <li>twitter_emotion_timeseries.html (percentage view)</li>
                <li>twitter_emotion_timeseries_stacked.html (count view)</li>
                <li>reddit_emotion_timeseries.html (percentage view)</li>
                <li>reddit_emotion_timeseries_stacked.html (count view)</li>
            </ul>
        </div>
    </body>
    </html>
    """

    filepath = os.path.join(save_dir, 'emotion_analysis_summary.html')
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(html_content)

    print(f" Summary report saved: emotion_analysis_summary.html")
    return filepath

# Generate summary
summary_path = create_summary_report(dfs, emotion_dir)
print(f"\n View summary: {summary_path}")

 Summary report saved: emotion_analysis_summary.html

 View summary: /content/drive/MyDrive/ClimateLens/visualizations/emotions/emotion_analysis_summary.html
